In [1]:
# Inspiration
# Remove the column that only contains missing values
# Do men or women speed more often?
# Does gender affect who gets searched during a stop?
# During a search, how often is the driver frisked?
# Which year had the least number of stops?
# When a man is pulled over, how often is it for speeding?
# How does drug activity change by time of day?
# Do most stops occur at night?

In [197]:
import pandas as pd
import numpy as np
from datetime import datetime,time, date
from dateutil import parser

In [70]:
police = pd.read_csv(r'C:\Users\godwi\OneDrive\Documents\Data Science\GitHub\Python-Pandas\Police\Police\police_project.csv')
police.head()

,stop_date,stop_time,county_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,2005-01-02,01:55,NaN,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
1,2005-01-18,08:15,NaN,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
2,2005-01-23,23:15,NaN,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
3,2005-02-20,17:15,NaN,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False
4,2005-03-14,10:00,NaN,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False


#  Checking the nature of the dataset

In [71]:
police.shape

(91741, 15)

In [72]:
police.info(verbose = False)  # check what data type is available

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91741 entries, 0 to 91740
Columns: 15 entries, stop_date to drugs_related_stop
dtypes: bool(2), float64(3), object(10)
memory usage: 9.3+ MB


In [74]:
# Total number of registered male and female drivers
police['driver_gender'].value_counts().sum()

86406

# Remove the column that only contains missing values

In [188]:
police.duplicated()== True

0        False
1        False
2        False
3        False
4        False
         ...  
91736    False
91737    False
91738    False
91739    False
91740    False
Length: 91741, dtype: bool

# Do men or women speed more often?

In [75]:
# ratio of male to female
police['driver_gender'].value_counts().to_frame()

,driver_gender
M,62895
F,23511


In [76]:
# % of male to female
round(police['driver_gender'].value_counts(1).to_frame(),3)
# Men speed more than women

,driver_gender
M,0.728
F,0.272


In [77]:
# check if there are any null values in driver_gender column ?
police.driver_gender.isnull().value_counts().to_frame()
# so we have 5,335 null values in the drive_gender column 

,driver_gender
False,86406
True,5335


# Does gender affect who gets searched during a stop?

In [78]:
police.loc[:, ['driver_gender', 'search_conducted']].groupby('driver_gender')\
.count().rename({'search_conducted' : 'Count_of_search'},axis = 1)

# Yes the male ate searched more than the female

,Count_of_search
driver_gender,
F,23511
M,62895


# During a search, how often is the driver frisked?

In [79]:
arrest = round(police.is_arrested.value_counts(100).to_frame(),2)
arrest
# Only 3% of the time , are drivers arrested

,is_arrested
False,0.97
True,0.03


# Which year had the least number of stops?

In [136]:
pd.DatetimeIndex(police.stop_date).year  # can use   ===  >  pd.DatetimeIndex(['stop_date']).year

Int64Index([2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005,
            ...
            2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015],
           dtype='int64', name='stop_date', length=91741)

In [137]:
least_year = police['stop_date'].to_frame()
least_year [:1]

,stop_date
0,2005-01-02


In [138]:
least_year['year'] = pd.DatetimeIndex(police['stop_date']).year

In [139]:
least_year.groupby('year').count().sort_values('stop_date', ascending = True).head(1)

,stop_date
year,
2005,2558


# When a man is pulled over, how often is it for speeding?

In [210]:
police[(police.violation.str.contains('Speeding')) & (police.driver_gender == 'M')]\
.violation.value_counts().to_frame().rename({'violation' :'no of violation by men'}, axis = 1)

,no of violation by men
Speeding,32979


In [233]:
police.loc[:, ['violation','driver_gender']].groupby ('driver_gender').get_group('M').value_counts()

violation            driver_gender
Speeding             M                32979
Moving violation     M                13020
Equipment            M                 8533
Other                M                 3627
Registration/plates  M                 2419
Seat belt            M                 2317
dtype: int64

# How does drug activity change by time of day?

In [153]:
tod_police = police [police['drugs_related_stop'] == True]\
.loc[:, ['stop_time', 'stop_duration' ,'drugs_related_stop']]
tod_police

,stop_time,stop_duration,drugs_related_stop
40,00:00,16-30 Min,True
41,00:00,30+ Min,True
118,01:20,30+ Min,True
139,00:20,16-30 Min,True
219,23:30,16-30 Min,True
...,...,...,...
91194,14:48,16-30 Min,True
91253,10:33,16-30 Min,True
91284,14:10,30+ Min,True
91672,11:05,16-30 Min,True
